In [ ]:
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
business = pd.read_csv('./resources/Listing_of_All_Businesses_20250310.csv')
crime = pd.read_csv('./resources/Crime_Data_20250310.csv')

business.info()

FileNotFoundError: [Errno 2] No such file or directory: './resources/Crime_Data_20250310.csv'

# Descripcción 

In [4]:
data.head(3)

,LOCATION ACCOUNT #,BUSINESS NAME,DBA NAME,STREET ADDRESS,CITY,ZIP CODE,LOCATION DESCRIPTION,MAILING ADDRESS,MAILING CITY,MAILING ZIP CODE,NAICS,PRIMARY NAICS DESCRIPTION,COUNCIL DISTRICT,LOCATION START DATE,LOCATION END DATE,LOCATION
0,0003217523-0001-2,MARIA OFELIA NUNEZ,LA CAMPANA HARDWARE,1017 W 88TH STREET,LOS ANGELES,90044-2119,1017 88TH 90044,1017 W 88TH STREET,LOS ANGELES,90044-2119,444130.0,Hardware stores,0,10/01/2020,NaN,"(33.9581, -118.2926)"
1,0003211603-0001-0,ONE STOP HOLISTIC HEALTH LLC,NaN,1506 W MANCHESTER AVENUE,LOS ANGELES,90047-5424,1506 MANCHESTER 90047,1506 W MANCHESTER AVENUE,LOS ANGELES,90047-5424,NaN,NaN,8,09/03/2020,NaN,"(33.9597, -118.3028)"
2,0003109317-0001-3,GINA BRADFORD,NaN,7847 CANBY AVENUE,RESEDA,91335-2096,7847 CANBY 91335,7847 CANBY AVENUE,RESEDA,91335-2096,NaN,NaN,4,01/01/2018,NaN,"(34.2133, -118.535)"


## Exploracion de datos nulos y faltantes

En este sentido el primer paso es explorar tanto los datos faltantes como sus distribuciones. Haciendo un análisis estadistico de los mismo en primer instancia.

Para empezar comenzaremos por familiarizarnos con el dataset y su estructura. Por tanto, comenzaremos por entender los datos que posee cada columna y como podemos trabajar con ellos.



| Campo                        | Descripción                                                                                                                                                                                                 |
|------------------------------|-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **LOCATION ACCOUNT #**       | Número de cuenta del lugar del negocio.                                                                                                                                                                     |
| **BUSINESS NAME**            | Nombre legal del negocio.                                                                                                                                                                                   |
| **DBA NAME**                 | Nombre(s) comercial(es) adicional(es), separados por barras verticales.                                                                                                                                     |
| **STREET ADDRESS**           | Dirección del negocio.                                                                                                                                                                                      |
| **CITY**                     | Ciudad del negocio.                                                                                                                                                                                         |
| **ZIP CODE**                 | Código postal del negocio.                                                                                                                                                                                  |
| **LOCATION DESCRIPTION**     | Descripción abreviada de la ubicación del negocio.                                                                                                                                                          |
| **MAILING ADDRESS**          | Dirección postal de la empresa.                                                                                                                                                                             |
| **MAILING CITY**             | Ciudad de la dirección postal.                                                                                                                                                                              |
| **MAILING ZIP CODE**         | Código postal de la dirección postal.                                                                                                                                                                       |
| **NAICS**                    | Código del Sistema de Clasificación Industrial de América del Norte (SCIAN).                                                                                                                                |
| **PRIMARY NAICS DESCRIPTION**| Descripción del código NAICS principal. Las descripciones marcadas como "(NAICS 1997)" aplican el código y la descripción designados en 1997, según lo dispuesto en la Ordenanza de Exención para Nuevas Empresas. |
| **COUNCIL DISTRICT**         | Número del distrito del consejo correspondiente a la dirección de la ubicación. "0" indica que la ubicación comercial está fuera de la ciudad.                                                              |
| **LOCATION START DATE**      | Fecha de inicio de la ubicación comercial.                                                                                                                                                                  |
| **LOCATION END DATE**        | Fecha de finalización de la ubicación comercial.                                                                                                                                                            |
| **LOCATION**                 | Latitud y longitud de la ubicación comercial.                                                                                                                                                               |


 se explorara en primera instancia la cantidad de datos nulos que tiene cada

In [9]:
data.isnull().sum()/len(data)

LOCATION ACCOUNT #           0.000000
BUSINESS NAME                0.000000
DBA NAME                     0.846318
STREET ADDRESS               0.000064
CITY                         0.000069
ZIP CODE                     0.000000
LOCATION DESCRIPTION         0.000077
MAILING ADDRESS              0.774126
MAILING CITY                 0.774120
MAILING ZIP CODE             0.774204
NAICS                        0.608865
PRIMARY NAICS DESCRIPTION    0.608865
COUNCIL DISTRICT             0.000000
LOCATION START DATE          0.000975
LOCATION END DATE            0.371569
LOCATION                     0.581329
dtype: float64